In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import xgboost as xgb

In [59]:
start = time.time()
df_train =pd.read_csv("training_set_VU_DM.csv")
end = time.time()
print("Train set loaded in ", end - start, " seconds.")

Train set loaded in  20.935261726379395  seconds.


In [60]:
df_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [61]:
df_train.shape

(4958347, 54)

In [62]:
df_train.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4706481
visitor_hist_adr_usd           4705359
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4640941
orig_destination_distance

In [63]:
df_train['booking_bool'].value_counts()

0    4819957
1     138390
Name: booking_bool, dtype: int64

In [64]:
counts = pd.DataFrame(df_train['prop_id'].value_counts(sort=False).reset_index())
counts.columns = ['prop_id', 'counts']

In [65]:
counts

,prop_id,counts
0,4098,64
1,8196,1
2,12294,19
3,16392,12
4,20490,6
...,...,...
129108,24564,16
129109,16376,50
129110,12282,10
129111,8188,2


In [66]:
clicks = pd.DataFrame(df_train.groupby('prop_id')['click_bool'].sum().reset_index())
clicks.columns = ['prop_id', 'clicks']


In [67]:
clicks

,prop_id,clicks
0,1,1
1,2,1
2,3,2
3,4,1
4,5,2
...,...,...
129108,140817,0
129109,140818,0
129110,140819,0
129111,140820,0


In [68]:
bookings = pd.DataFrame(df_train.groupby('prop_id')['booking_bool'].sum().reset_index())
bookings.columns = ['prop_id', 'bookings']

In [69]:
bookings

,prop_id,bookings
0,1,0
1,2,1
2,3,2
3,4,1
4,5,0
...,...,...
129108,140817,0
129109,140818,0
129110,140819,0
129111,140820,0


In [70]:
temp = pd.merge(counts, clicks, left_on='prop_id', right_on='prop_id')
df_rates = pd.merge(temp, bookings, left_on='prop_id', right_on='prop_id')

In [71]:
df_rates

,prop_id,counts,clicks,bookings
0,4098,64,1,1
1,8196,1,0,0
2,12294,19,1,1
3,16392,12,0,0
4,20490,6,0,0
...,...,...,...,...
129108,24564,16,1,0
129109,16376,50,4,3
129110,12282,10,1,1
129111,8188,2,0,0


0    4736468
5     138390
1      83489
Name: score, dtype: int64

0    27
5     1
Name: score, dtype: int64

# Handle Missing Values

In [75]:
#fill NaNs in 'gross_bookings_usd with 0.
df_train.loc[df_train['gross_bookings_usd'].isnull(), 'gross_bookings_usd'] = 0

In [76]:
#fill NaNs with 0 for compet
comp_descr = {i: 0 for i in df_train.columns[27:51]}
df_train.fillna(comp_descr, inplace=True)

In [78]:
df_train[['visitor_hist_starrating','prop_starrating']]

,visitor_hist_starrating,prop_starrating
0,NaN,3
1,NaN,4
2,NaN,3
3,NaN,2
4,NaN,4
...,...,...
4958342,NaN,3
4958343,NaN,3
4958344,NaN,3
4958345,NaN,3


In [79]:
df_train['visitor_hist_starrating'].value_counts()

4.00    23416
3.00    20879
3.50    17088
2.50    10534
2.00     9157
        ...  
2.01       16
1.96       11
1.92        7
4.65        6
4.96        6
Name: visitor_hist_starrating, Length: 312, dtype: int64

In [80]:
df_train['visitor_hist_starrating'].fillna(0, inplace=True)
df_train['visitor_hist_adr_usd'].fillna(0, inplace=True)

In [85]:
df_train['prop_review_score'].fillna(0, inplace=True)
df_train['prop_location_score2'].fillna(0, inplace=True)

In [92]:
df_train['orig_destination_distance'].mean()

1301.2344059971783

If the mean is greater than the median, the distribution is positively skewed.

In [93]:
df_train['orig_destination_distance'].median()

386.6

In [94]:
df_train['orig_destination_distance'].fillna(df_train['orig_destination_distance'].mean(), inplace=True)

In [96]:
df_train['srch_query_affinity_score'].describe()

count    317406.000000
mean        -24.146418
std          15.743238
min        -326.567500
25%         -30.774775
50%         -20.451300
75%         -13.350625
max          -2.494100
Name: srch_query_affinity_score, dtype: float64

In [97]:
df_train['srch_query_affinity_score'].fillna(-330, inplace=True)

In [98]:
#Check if any more missing values in the dataset
df_train.isnull().sum()

srch_id                        0
date_time                      0
site_id                        0
visitor_location_country_id    0
visitor_hist_starrating        0
visitor_hist_adr_usd           0
prop_country_id                0
prop_id                        0
prop_starrating                0
prop_review_score              0
prop_brand_bool                0
prop_location_score1           0
prop_location_score2           0
prop_log_historical_price      0
position                       0
price_usd                      0
promotion_flag                 0
srch_destination_id            0
srch_length_of_stay            0
srch_booking_window            0
srch_adults_count              0
srch_children_count            0
srch_room_count                0
srch_saturday_night_bool       0
srch_query_affinity_score      0
orig_destination_distance      0
random_bool                    0
comp1_rate                     0
comp1_inv                      0
comp1_rate_percent_diff        0
comp2_rate

# Feature Engineering

Create a column called score which would be 
df['score'] = 4 * df.booking_bool + df.click_bool

In [72]:
df_train['score'] = 4 * df_train.booking_bool + df_train.click_bool

In [83]:
df_train[df_train['srch_id'] == 1].score.value_counts()

0    27
5     1
Name: score, dtype: int64

In [81]:
df_train['starrating_diff'] = np.abs(df_train['visitor_hist_starrating'] - df_train['prop_starrating'])
df_train['usd_diff'] = np.abs(df_train['visitor_hist_adr_usd'] - df_train['price_usd'])

In [82]:
df_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,score,starrating_diff,usd_diff
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,0.0,0.0,0.0,0.0,0,0.0,0,0,3.0,104.77
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,0.0,0.0,0.0,0.0,0,0.0,0,0,4.0,170.74
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,0.0,0.0,0.0,0.0,0,0.0,0,0,3.0,179.80
3,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,27348,2,4.0,...,0.0,-1.0,0.0,5.0,0,0.0,0,0,2.0,602.77
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,0.0,0.0,0.0,0.0,0,0.0,0,0,4.0,143.58
